# Ingeniería de Características (Feature Engineering)
## Proyecto de Mantenimiento Predictivo - Moto-Compresores

Este notebook contiene la creación y selección de características para el modelo predictivo.

### Objetivos:
- Crear características derivadas
- Generar indicadores de tendencia
- Seleccionar características relevantes
- Crear variables de degradación

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

## 1. Carga de Datos Procesados

In [ ]:
# TODO: Cargar datos desde data/processed

## 2. Características Temporales

In [ ]:
# TODO: Crear características basadas en tiempo
# TODO: Ventanas móviles y estadísticas agregadas
# TODO: Tendencias y derivadas

## 3. Indicadores de Degradación

In [ ]:
# TODO: Crear indicadores de salud del equipo
# TODO: Métricas de eficiencia
# TODO: Ratios y relaciones entre variables

## 4. Selección de Características

In [ ]:
# TODO: Análisis de importancia de características
# TODO: Eliminación de características redundantes
# TODO: Selección basada en correlación

## 5. Dataset Final de Características

In [ ]:
# TODO: Consolidar dataset final
# TODO: Guardar características seleccionadas